In [ ]:
#Pop!_OS by System76

#Jupyter-notebook
sudo apt-get update
sudo apt-get install -y jupyter-notebook
jupyter-notebook --generate-config
sudo touch /etc/rc.local
sudo chmod u+x /etc/rc.local
echo '
# value on error.
#
# In order to enable or disable this script just change the execution
# bits.
#
# By default this script does nothing.

#-u user

sudo env "PATH=\$PATH" nohup jupyter-notebook \
--allow-root \
--ip 0.0.0.0 --port 8888 \
--no-browser \
--notebook-dir="/home/user/notebooks" \
--NotebookApp.token="" \
--NotebookApp.password="" \
& 
exit 0' | sudo tee -a /etc/rc.local

# nopasswd sudo
echo "$USER    ALL=(ALL) NOPASSWD:ALL" | sudo tee -a /etc/sudoers

In [ ]:
# Repository
!sudo add-apt-repository universe
!sudo add-apt-repository multiverse
!sudo apt update

!sudo apt update
!sudo apt autoremove -y

In [ ]:
# disable animation GNOME3
#https://blog.christophersmart.com/2014/03/15/disable-animations-in-gnome-3-for-older-hardware/
!gsettings set org.gnome.desktop.interface enable-animations false

#no hibernate

# Browser
## Vivaldi
!echo "deb http://repo.vivaldi.com/stable/deb/ stable main" | sudo tee /etc/apt/sources.list.d/vivaldi.list > /dev/null
!wget -O - http://repo.vivaldi.com/stable/linux_signing_key.pub | sudo apt-key add -
!sudo apt update && sudo apt install vivaldi-stable -y
#move backup
#! cp  
## Google Chrome
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
!sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'
!sudo apt update
!sudo apt install google-chrome-stable -y

## Microsoft Edge

# VSCode




In [ ]:
# Auto login
# https://linuxconfig.org/how-to-enable-automatic-login-on-ubuntu-20-04-focal-fossa
# GDM3 (Ubuntu, Pop!OS)
!sudo sed -i 's/#  AutomaticLoginEnable = true/AutomaticLoginEnable = true/g' /etc/gdm3/custom.conf
!sudo sed -i "s/#  AutomaticLogin = user1/AutomaticLogin = $(echo $USER)/g" /etc/gdm3/custom.conf

In [ ]:
#https://linuxconfig.org/how-to-enable-autologin-on-kali-linux
cat /etc/gdm3/daemon.conf
sudo sed -i 's|# AutomaticLoginEnable = true|AutomaticLoginEnable = true|g' /etc/gdm3/daemon.conf
sudo sed -i 's|# AutomaticLogin = root|AutomaticLogin = root|g' /etc/gdm3/daemon.conf

# LightDM (Kali, WSL)
sudo apt-get install -y xfce4 xfce4-places-plugin xfce4-goodies
update-alternatives --config x-session-manager
sudo apt-get install -y kali-undercover
#https://linuxconfig.org/how-to-enable-autologin-on-kali-linux
cat /etc/lightdm/lightdm.conf

sudo sed -i 's|#autologin-guest|autologin-guest|g' /etc/lightdm/lightdm.conf
sudo sed -i 's|#autologin-user=|autologin-user=kali|g' /etc/lightdm/lightdm.conf
sudo sed -i 's|#autologin-user-timeout|autologin-user-timeout|g' /etc/lightdm/lightdm.conf

cat /etc/pam.d/lightdm-autologin
sudo sed -i 's|auth required pam_succeed|###auth required pam_succeed|g' /etc/pam.d/lightdm-autologin

In [ ]:
# Install
## Drivers

In [ ]:
### RTL8812BU
# https://github.com/cilynx/rtl88x2bu
cd ~
git clone https://github.com/cilynx/rtl88x2bu.git
cd rtl88x2bu
VER=$(sed -n 's/\PACKAGE_VERSION="\(.*\)"/\1/p' dkms.conf)
sudo rsync -rvhP ./ /usr/src/rtl88x2bu-${VER}
sudo dkms add -m rtl88x2bu -v ${VER}
sudo dkms build -m rtl88x2bu -v ${VER}
sudo dkms install -m rtl88x2bu -v ${VER}
sudo modprobe 88x2bu

In [ ]:
### RTL8188EUS
# AIRCRACK-NG/RTL8188EUS_TLWN722N
# https://github.com/aircrack-ng/rtl8188eus
sudo apt update
sudo apt install bc -y
cd ~
git clone https://github.com/aircrack-ng/rtl8188eus
cd rtl8188eus
#cp realtek_blacklist.conf /etc/modprobe.d/
#echo "blacklist r8188eu" > /etc/modprobe.d/realtek.conf
printf "blacklist r8188eu" | sudo tee -a /etc/modprobe.d/realtek.conf
make && sudo make install

#sudo mkdir -p /etc/modprobe.d/
#sudo touch /etc/modprobe.d/rtl8188eus.conf
#echo "options rtl8188eus rtw_power_mgnt=0 rtw_enusbss=0" | sudo tee /etc/modprobe.d/rtl8188eus.conf
echo "


[device]
wifi.scan-rand-mac-address=no

[ifupdown]
managed=false

[connection]
wifi.powersave=0

[main]
plugins=keyfile

[keyfile]
unmanaged-devices=mac:A7:A7:A7:A7:A7" | sudo tee -a /etc/NetworkManager/NetworkManager.conf
sudo rmmod 8188eu && sudo insmod 8188eu.ko


In [ ]:
#AIRMON
sudo airmon-ng check-kill
sudo ip link set wlan0 down
sudo iw dev wlan0 set type monitor

In [ ]:
### RTL8188FTV
#InstallRTL8188FTV
#https://askubuntu.com/questions/1062402/cant-find-wifi-drivers-for-0bdaf179-realtek-semiconductor-corp
cd ~
sudo apt-get install build-essential git dkms linux-headers-$(uname -r) -y
git clone https://github.com/kelebek333/rtl8188fu
sudo dkms add ./rtl8188fu
sudo dkms build rtl8188fu/1.0
sudo dkms install rtl8188fu/1.0
sudo cp ./rtl8188fu/firmware/rtl8188fufw.bin /lib/firmware/rtlwifi/

In [ ]:

### Intel AC8265NGW
### Nvidia
#AIRCRACK-NG/RTL8812AU
sudo apt-get update
sudo apt-get install -y build-essential bc libelf-dev linux-headers-`uname -r`
cd ~
git clone -b v5.6.4.2 https://github.com/aircrack-ng/rtl8812au.git
cd rtl8812au
sudo ./dkms-install.sh
#sudo ./dkms-remove.sh

#For Raspberry (RPI)
#sudo apt-get install bc raspberrypi-kernel-headers

#Then run this step to change platform in Makefile, For RPI 1/2/3/ & 0/Zero:
#sed -i 's/CONFIG_PLATFORM_I386_PC = y/CONFIG_PLATFORM_I386_PC = n/g' Makefile
#sed -i 's/CONFIG_PLATFORM_ARM_RPI = n/CONFIG_PLATFORM_ARM_RPI = y/g' Makefile

#But for RPI 3B+ & 4B you will need to run those below which builds the ARM64 arch driver:
#sed -i 's/CONFIG_PLATFORM_I386_PC = y/CONFIG_PLATFORM_I386_PC = n/g' Makefile
#sed -i 's/CONFIG_PLATFORM_ARM64_RPI = n/CONFIG_PLATFORM_ARM64_RPI = y/g' Makefile

#In addition, if you receive an error message about unrecognized command #line option ‘-mgeneral-regs-only’ (i.e., Raspbian Buster), you will need to run the following commands:
#sed -i 's/^dkms build/ARCH=arm dkms build/' dkms-install.sh
#sed -i 's/^MAKE="/MAKE="ARCH=arm\ /' dkms.conf


In [1]:
# Others
!sudo apt install -y anki
!sudo apt install -y autokey-gtk
!sudo apt install -y handbrake 

# Pacman
!sudo apt install -y flatpak 
!sudo apt install -y snapd

!sudo snap install bing-wall
!sudo snap install spotify
!sudo snap install discord
!sudo snap install anbox




Reading package lists... Done
Building dependency tree       
Reading state information... Done
anki is already the newest version (2.1.15+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 319 not upgraded.
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google.list:1
W: Target Packages (main/binary-all/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google.list:1
W: Target Translations (main/i18n/Translation-en_US) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google.list:1
W: Target Translations (main/i18n/Translation-en) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google.list:1
W: Target DEP-11 (main/dep11/Components-amd64.yml) is configured multiple times in /etc/apt/sources.list.d/google-

In [ ]:
# Virtualisation
# Qemu-KVM
sudo apt-get install qemu-system qemu-utils python3 python3-pip -y
sudo apt-get install libvirt-bin libvirt-doc -y
sudo apt install qemu-kvm libvirt-clients libvirt-daemon-system bridge-utils virt-manager -y
sudo apt-get install libvirt-daemon-system-sysv -y
sudo service libvirtd start
sudo adduser username libvirt
sudo adduser username libvirt-qemu



In [ ]:
# Virtualbox
sudo apt install -y virtualbox

In [ ]:
# Wine-Stable
!cn=$(awk -F= '/VERSION_CODENAME=/ {gsub(/"/,""); print $2}' /etc/os-release)
!sudo dpkg --add-architecture i386
!wget -nc https://dl.winehq.org/wine-builds/winehq.key
!sudo apt-key add winehq.key
!sudo apt-add-repository -y "deb https://dl.winehq.org/wine-builds/ubuntu/ $(awk -F= '/VERSION_CODENAME=/ {gsub(/"/,""); print $2}' /etc/os-release) main"
!sudo apt update
#!sudo apt install -y python3-pil python3-setproctitle cabextract p7zip fluid-soundfont-gs mesa-utils python3-evdev libwine-development winetricks libc6-i386 lib32gcc1 lib32gcc-s1
#!sudo apt-get install -y libgnutls30:i386 libldap-2.4-2:i386 libgpg-error0:i386 libxml2:i386 libasound2-plugins:i386 libsdl2-2.0-0:i386 libfreetype6:i386 libdbus-1-3:i386 libsqlite3-0:i386
!sudo apt install --install-recommends wine-stable

# CrossOver
# Office 2016

In [ ]:
#Lutris
!sudo add-apt-repository -y "deb http://archive.ubuntu.com/ubuntu/ focal main"
!sudo add-apt-repository -y ppa:lutris-team/lutris
!sudo apt-get update
!sudo apt-get install lutris -y

In [ ]:
#Node-red
#selenium webdriver
#chromedriver

In [ ]:
#Kali-tools
sudo apt install -y aircrack-ng
sudo apt-get install -y hashcat

In [ ]:
### Brew
sudo apt update
sudo apt-get install build-essential curl file git -y
sudo apt install -y ruby
git clone https://github.com/Homebrew/linuxbrew.git ~/.linuxbrew
export PATH="$HOME/.linuxbrew/bin:$PATH"
export MANPATH="$HOME/.linuxbrew/share/man:$MANPATH"
export INFOPATH="$HOME/.linuxbrew/share/info:$INFOPATH"
hash -r

test -d ~/.linuxbrew && eval $(~/.linuxbrew/bin/brew shellenv)
test -d /home/linuxbrew/.linuxbrew && eval $(/home/linuxbrew/.linuxbrew/bin/brew shellenv)
test -r ~/.bash_profile && echo "eval \$($(brew --prefix)/bin/brew shellenv)" >>~/.bash_profile
echo "eval \$($(brew --prefix)/bin/brew shellenv)" >>~/.profile



In [ ]:
INSTALLUBUNTU18.04

#EXFATSUPPORT
sudo add-apt-repository universe
sudo apt update
sudo apt install -y exfat-fuse exfat-utils

sudo apt-get install -y \
xclip \
openvpn

sudo apt-get install -y \
linux-tools-generic \
linux-cloud-tools-generic \


#USBIP
http://www.linux-magazine.com/Issues/2018/208/Tutorial-USB-IP
#SERVERSIDE
sudo modprobe usbip-core
sudo modprobe usbip-host
sudo usbipd
sudo usbipd -D
sudo usbip list -l
sudo usbip bind -b 1-2
#sudo usbip unbind -b 1-2

#CLIENTSIDE
sudo modprobe usbip-core
sudo modprobe vhci-hcd
sudo usbip list -r <server>
sudo usbip list -r 192.168.1.24

sudo ln -s /usr/lib/flfl
  linux-tools-<version number>/usbip /usr/sbin/
sudo usbip attach -r 192.168.1.24 -b 1-1
lsusb

#DETACHING
sudo usbip port
sudo usbip detach -p 01


#UBUNTUDRIVERS
sudo apt-get install -y ubuntu-drivers
sudo ubuntu-drivers autoinstall

#INSTALLVBOX
sudo apt-get install -y virtualbox
sudo modprobe vboxdrv


#INSTALLWIN10THEME
cd ~/Documents/Docs/mlinuxproject/usbboot/Documents3/TW
wget https://raw.githubusercontent.com/bill-mavromatis/gnome-layout-manager/master/layoutmanager.sh
chmod +x layout*
./layoutmanager.sh


https://github.com/B00merang-Project/TransPack/archive/master.zip
git clone https://github.com/B00merang-Project/TransPack/archive/master
cd TransPack
chmod +x install.sh
./install.sh

#cd ~/Downloads
cd ~
git clone https://github.com/B00merang-Project/Windows-10.git
#unzip Windows-10-2.0.1.zip
mkdir ~/.themes
mv ~/Windows-10 ~/.themes
#sudo apt-get install -y unity-tweak-tool
#unity-tweak-tool
sudo apt-get install -y gnome-tweaks












In [ ]:
#MIRACAST
sudo add-apt-repository ppa:thopiekar/miraclecast
sudo apt-get update
sudo apt-get install miraclecast

In [ ]:
##NETWORKING

In [ ]:
#MOBILEHOTSPOT
git clone
cd ~/create_ap
sudo create_ap wlxd0374534a264 wlx1cbfce35c3c1 WIFI_A8233 Mylo2012


In [ ]:

#SETSTATICINET_BASH
#https://websiteforstudents.com/switch-static-ip-address-ubuntu-17-04-17-10/
sudo sed -i 's|inet dhcp| inet static
address 10.150.141.79
netmask 255.255.240.0
#network 192.168.0.0
#broadcst 192.168.0.255
gateway 10.150.128.1
dns-nameservers 10.132.160.101
|g' /etc/network/interfaces

In [ ]:
#STATICIPUBUNTU
#1
sudo nano /etc/netplan/01-netcfg.yaml
sudo netplan apply

#2
sudo nano /etc/network/interfaces
auto eno1
iface eno1 inet dhcp
#iface eno1 inet static
#address 10.150.141.79
#netmask 255.255.240.0
#gateway 10.150.128.1
#dns-nameservers 10.132.160.101